## Load data and libraries

In [2]:
import pandas as pd
import os
import numpy as np 

In [3]:
# Directory where your CSV files are located
directory = '/Users/alejandrovillanuevalledo/Documents/GitHub/Hong-Kong_project/CSV'

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):  # Check if the file is a CSV file
        file_path = os.path.join(directory, filename)  # Get the full file path
        # Read the CSV file and create a DataFrame with a variable name based on the filename (without extension)
        dataframe_name = os.path.splitext(filename)[0]  # Extract the filename without extension
        globals()[dataframe_name] = pd.read_csv(file_path)

# Now you have separate DataFrame variables named after the filenames (without extension)
# For example, if you have a file 'calendar.csv', you can access its DataFrame using 'calendar' variable
# Similarly, if you have a file 'listings.csv', you can access its DataFrame using 'listings' variable


In [ ]:
#listing_2 = pd.read_csv('/Users/alejandrovillanuevalledo/Documents/GitHub/Hong-Kong_project/CSV/listings-2.csv')

In [ ]:
#listing_2.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

In [4]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [9]:
colums_to_drop = ['listing_url', 'scrape_id', 'last_scraped', 'source','picture_url','host_url', 'host_thumbnail_url', 'host_picture_url','neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
'longitude','bathrooms', 'amenities','calendar_updated','calendar_last_scraped']
listings = listings.drop(columns= colums_to_drop)

In [13]:

# Find columns that are unique to df1
unique_to_df1 = listings.columns.difference(listing_2.columns)

# Find columns that are unique to df2
unique_to_df2 = listing_2.columns.difference(listings.columns)

# Print columns that are not present in both DataFrames
print("Columns unique to df1:", list(unique_to_df1))
print("Columns unique to df2:", list(unique_to_df2))


Columns unique to df1: ['accommodates', 'amenities', 'availability_30', 'availability_60', 'availability_90', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'calendar_last_scraped', 'calendar_updated', 'description', 'first_review', 'has_availability', 'host_about', 'host_acceptance_rate', 'host_has_profile_pic', 'host_identity_verified', 'host_is_superhost', 'host_listings_count', 'host_location', 'host_neighbourhood', 'host_picture_url', 'host_response_rate', 'host_response_time', 'host_since', 'host_thumbnail_url', 'host_total_listings_count', 'host_url', 'host_verifications', 'instant_bookable', 'last_scraped', 'listing_url', 'maximum_maximum_nights', 'maximum_minimum_nights', 'maximum_nights', 'maximum_nights_avg_ntm', 'minimum_maximum_nights', 'minimum_minimum_nights', 'minimum_nights_avg_ntm', 'neighborhood_overview', 'neighbourhood_cl

In [25]:
calendar.nunique()

listing_id        7029
date               366
available            2
price              920
adjusted_price       0
minimum_nights      68
maximum_nights     103
dtype: int64

In [26]:
#del neighbourhoods

## Clean Data

In [15]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7029 entries, 0 to 7028
Data columns (total 59 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            7029 non-null   int64  
 1   name                                          7029 non-null   object 
 2   description                                   0 non-null      float64
 3   neighborhood_overview                         2131 non-null   object 
 4   host_id                                       7029 non-null   int64  
 5   host_name                                     7029 non-null   object 
 6   host_since                                    7029 non-null   object 
 7   host_location                                 4593 non-null   object 
 8   host_about                                    4472 non-null   object 
 9   host_response_time                            6302 non-null   o

In [18]:
listings.duplicated().sum()

0

In [14]:
listings.isnull().sum()#drop description & bedrooms & license, neighborhood overview & host about turn binomial, 

id                                                 0
name                                               0
description                                     7029
neighborhood_overview                           4898
host_id                                            0
host_name                                          0
host_since                                         0
host_location                                   2436
host_about                                      2557
host_response_time                               727
host_response_rate                               727
host_acceptance_rate                             551
host_is_superhost                                 16
host_neighbourhood                               443
host_listings_count                                0
host_total_listings_count                          0
host_verifications                                 0
host_has_profile_pic                               0
host_identity_verified                        

In [21]:
listings.isnull().all(axis = 1).sum()

0

In [29]:
pd.set_option('display.max_columns', None)
listings[listings['first_review'].isnull()]

,id,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
6,263081,Rental unit in Mid-level · 3 bedrooms · 6 beds...,NaN,"We are in the most luxury area of HK, conduit ...",1370155,Chuster,2011-11-04,Hong Kong,Hi! My name is Peter and I have been living in...,a few days or more,0%,0%,f,The Peak,1,1,"['email', 'phone', 'work_email']",t,t,"Mid-level, Hong Kong",Private room in rental unit,Private room,8,2.5 baths,NaN,6.0,"$22,281.00",10,30,10,10,30,30,10.0,30.0,t,5,35,65,340,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
39,894221,Rental unit in Mong Kok · 1 bedroom · 1 bed,NaN,NaN,4584648,Ivy,2013-01-04,Hong Kong,I have a few rooms available. \n \nLots of roo...,within a few hours,100%,32%,f,Mong Kok,430,503,"['email', 'phone']",t,t,NaN,Private room in rental unit,Private room,1,NaN,NaN,1.0,$180.00,30,730,30,30,730,730,30.0,730.0,t,30,60,90,365,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,398,12,378,8,NaN
40,894224,Rental unit in Hong Kong · 1 bedroom · 1 bed,NaN,NaN,4584648,Ivy,2013-01-04,Hong Kong,I have a few rooms available. \n \nLots of roo...,within a few hours,100%,32%,f,Mong Kok,430,503,"['email', 'phone']",t,t,NaN,Private room in rental unit,Private room,1,NaN,NaN,1.0,$140.00,31,1125,31,31,1125,1125,31.0,1125.0,t,29,59,89,364,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,398,12,378,8,NaN
53,1036146,Rental unit in Discovery Bay · 3 bedrooms · 3 ...,NaN,"Golf club, residence club, Disnyland, Nyoung P...",2376514,Sun-Ok,2012-05-15,Hong Kong,"I am mum of two grown up Kids , and having m...",within a day,50%,75%,f,Sheung Wan,4,12,"['email', 'phone']",t,t,"Discovery Bay, Hong Kong",Entire rental unit,Entire home/apt,6,2 baths,NaN,3.0,"$1,857.00",90,90,90,90,90,90,90.0,90.0,t,30,60,90,365,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,4,3,1,0,NaN
54,1046393,Rental unit in Wan Chai · 1 bedroom · 1 bed · ...,NaN,NaN,4584648,Ivy,2013-01-04,Hong Kong,I have a few rooms available. \n \nLots of roo...,within a few hours,100%,32%,f,Mong Kok,430,503,"['email', 'phone']",t,t,NaN,Private room in rental unit,Private room,2,1 bath,NaN,1.0,$180.00,30,1125,30,30,1125,1125,30.0,1125.0,t,30,60,90,365,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,398,12,378,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7024,1049537442224124625,Bed and breakfast in Hong Kong · ★New · 1 bedr...,NaN,NaN,6763017,Rina,2013-06-05,Hong Kong,Éclat Habitat passionate about service apartme...,within a few hours,97%,73%,f,Tai Tam,49,50,"['email', 'phone']",t,t,NaN,Private room in bed and breakfast,Private room,2,1 bath,NaN,1.0,"$1,737.00",1,365,1,1,365,365,1.0,365.0,t,29,59,89,269,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,42,19,23,0,NaN
7025,1049697550164824387,Rental unit in Hong Kong · ★New · 1 bedroom · ...,NaN,NaN,353326613,S